In [1]:
import pandas as pd
import numpy as np
from itertools import combinations

In [2]:
f = pd.read_csv('D_data.csv')
f

,x0,x1,x2,x3
0,1,3,4,1
1,1,1,2,1
2,1,1,1,1
3,2,4,1,1
4,2,4,1,1
5,2,4,1,1
6,3,2,2,1
7,3,1,2,1


- to show that it is not the case thax =(x0,x1) it is enough to find one value of x0 such that the assignments with it have different values of x1
the dependence arguments and dependent variable are named left,right respectively
- arguments 
    1. dataframe
    2. left - string/list of strings (names of columns from the dataframe)
    3. right - string (names of column from the dataframe)

In [3]:
def check_dependence(df,left_cols,right_col):
    # find columns from the dataframe present in the atomic formula of dependence logic
    if type(left_cols) == str:
        list_of_cols = [left_cols,right_col]
    else:
        list_of_cols = left_cols.copy()
        list_of_cols.append(right_col)
        
    #group the data by all possible combinations of the left_cols values
    g = df[list_of_cols].groupby(left_cols)[right_col].nunique()

    #check whether one combination has multiple possible values in right_col
    if len(g[g > 1])==0:
        return True
    else:
        return False

In [4]:
# check_dependence(f,'x0','x1') #results in False
# check_dependence(f[f['x0']==2],['x0'],'x1') #results in True
check_dependence(f,['x0','x1'],'x2') #results in False

False

In [5]:
def generate_potential_depedences(columns,max_number_of_left_cols):
    dependences_to_check = []
    for r in columns: #iterate over right columns
        right_col = r
        potential_left_cols = list(columns.drop(right_col))
        for i in np.arange(start=1,stop=max_number_of_left_cols+1,step=1): #iterate over possible numbers of left columns
            left_cols = list(combinations(potential_left_cols,r = int(i))) #iterate over all possible combinations of left columns
            #agg left_cols and right_col into a list
            for l in left_cols:
                dependences_to_check.append([list(l),right_col])
                #dependences_to_check_df['left_cols'] = list(l)
                #dependences_to_check_df['right_col'] = right_col
    return dependences_to_check


In [6]:
def find_dependencies(df,max_number_of_left_cols=1):
    potential_dependencies = generate_potential_depedences(df.columns,max_number_of_left_cols)
    print(potential_dependencies)
    potential_dependencies_df =pd.DataFrame(potential_dependencies, columns=['left_cols', 'right_col'])   
    potential_dependencies_df['result'] = potential_dependencies_df.apply(lambda x: check_dependence(df,x['left_cols'],x['right_col']),axis=1)
    return potential_dependencies_df[potential_dependencies_df['result']==True]

In [7]:
x = find_dependencies(f,2)

[[['x1'], 'x0'], [['x2'], 'x0'], [['x3'], 'x0'], [['x1', 'x2'], 'x0'], [['x1', 'x3'], 'x0'], [['x2', 'x3'], 'x0'], [['x0'], 'x1'], [['x2'], 'x1'], [['x3'], 'x1'], [['x0', 'x2'], 'x1'], [['x0', 'x3'], 'x1'], [['x2', 'x3'], 'x1'], [['x0'], 'x2'], [['x1'], 'x2'], [['x3'], 'x2'], [['x0', 'x1'], 'x2'], [['x0', 'x3'], 'x2'], [['x1', 'x3'], 'x2'], [['x0'], 'x3'], [['x1'], 'x3'], [['x2'], 'x3'], [['x0', 'x1'], 'x3'], [['x0', 'x2'], 'x3'], [['x1', 'x2'], 'x3']]
